# Module 4 Homework

## More Walmart Cleaning

The file **Walmart_United_States\_&\_Canada.csv** contains data on all
Walmarts, Sam's Clubs and Murphy USA gas/diesel in the USA and Canada.
Note that Gas/Diesel and No Over Night Parking (NOP) are indicated if
known. These data can be obtained from the site
<http://www.poi-factory.com/node/25560>.


**Before you start.** You started cleaning this data set in Activity 4.1.  Start by copying over your code and fixing the encoding issue.

In [2]:
!pip install more_itertools

In [12]:
#pd.set_option("display.max_cdwidth",None)
import pandas as pd
from dfply import *
from more_dfply import case_when, ifelse, fix_names, recode
from more_dfply.facets import text_facet, text_filter
walmart = pd.read_csv("./Walmart_United_States_&_Canada.csv", names = ['Lat','Lon','Store','Address'])
walmart.head()

(walmart
>> select(X.Store)
>> filter_by(text_filter(X.Store, 'Gas'))
>> filter_by(text_filter(X.Store, 'Gas/Diesel'))
>> head())

,Store
70,"Murphy: USA; #7687,Gas/Diesel,"
71,"Walmart Supercenter; #0423,Gas/Diesel,"
75,"Walmart Supercenter; #1091,Gas/Diesel,"
76,"Murphy: USA; #7394,Gas/Diesel,"
79,"Walmart Supercenter; #0306,Gas/Diesel,"


In [13]:
new_store_name = {'murphy: USA':'Murphy','Murphy: USA':'Murphy','Walmart Supercentre':'Walmart Supercenter','WM Nbrhd Mkt':'Walmart Neighborhood Market',
            'Wm Nbrhd Mkt':'Walmart Neighborhood Market','wm Nbrhd Mkt':'Walmart Neighborhood Market','Wm Pharmacy/Clinic':'Walmart Pharmacy/Clinic',
            'Walmart':'Walmart',"Sam's Club":'Sams Club','Walmart Pickup & Fuel':'Walmart Pickup & Fuel','Walmart Pickup':'Walmart Pickup',
            'Walmart Fuel Center':'Walmart Fuel Center','Walmart Supercenter':'Walmart Supercenter'}
import numpy as np
walmart2 = (walmart

>> mutate(fuel = case_when((text_filter(X.Store, 'Gas/Diesel'),
                                                "Gas/Diesel"),
                          (text_filter(X.Store, 'Gas'),
                                                "Gas"),
                                                (True, "None")
                                                ),
          Store = X.Store.replace('Gas(/Diesel)?,', '', regex = True),
)
>> mutate(storetype = case_when((text_filter(X['Store'], 'Walmart( Supercentre| Supercenter| Pickup(& Fuel)?)?;(\s)?#\d{3,4},',regex=True),
                      recode(X.Store.str.split(";").str.get(0), new_store_name)),
                      (text_filter(X['Store'],'USA; #\d{4},$',regex=True),
                      recode(X.Store.str.split(";").str.get(0), new_store_name)),
                      (text_filter(X['Store'],'Wm (Nbrhd Mkt|Pharmacy/Clinic)(; |,)#\d{4},',regex=True),
                      recode(X.Store.str.split(";|,").str.get(0), new_store_name)),
                      (text_filter(X['Store'],'\w{3}\W\w Club; #\d{4},', regex=True),
                      recode(X.Store.str.split(";").str.get(0), new_store_name)),
                      (text_filter(X['Store'],'Walmart (Fuel Center|Supercenter),#\d{4},', regex=True),
                      recode(X.Store.str.split(";").str.get(0), new_store_name)),
                      (True,recode(X.Store.str.split(";").str.get(0), new_store_name))))
>> mutate(storello= case_when((text_filter(X['Store'],'#\d{3,4},$', regex=True),
          X.Store.str.split("#").str.get(1).str.slice(stop=4)),
          (True,'fudge')))
)
walmart2.head()

/Users/paytonsimmons/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/more_dfply/facets.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return col.str.contains(pattern, case=case, regex=regex, na=na)
/Users/paytonsimmons/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/more_dfply/facets.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return col.str.contains(pattern, case=case, regex=regex, na=na)
/Users/paytonsimmons/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/more_dfply/facets.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return col.str.contains(pattern, case=case, regex=regex, na=na)


,Lat,Lon,Store,Address,fuel,storetype,storello
0,-114.005671,51.262567,"Walmart Supercentre; #1050,","2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-...",None,Walmart Supercenter,1050
1,-111.900542,50.577939,"Walmart Supercentre; #3658,","917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111",None,Walmart Supercenter,3658
2,-114.039133,51.107253,"Walmart Supercentre; #3013,","1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,(40...",None,Walmart Supercenter,3013
3,-114.138488,51.040871,"Walmart Supercentre; #3009,","1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,(403) ...",Gas,Walmart Supercenter,3009
4,-114.028603,50.930551,"Walmart; #1144,","1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,...",None,Walmart,1144


In [14]:
(walmart
>> select(X.Store)
>> filter_by(~text_filter(X.Store, 'Gas/Diesel'))
>> filter_by(~text_filter(X.Store, 'Gas'))
>> mutate(Store = X.Store.replace('Gas(/Diesel)?,', '', regex=True))
>> filter_by(~text_filter(X.Store, 'Walmart( Supercentre| Supercentre)?; #\d{4},', regex=True))
>> mutate(Store = X.Store.replace(r'Wm (.*)', r'Walmart \1', regex=True)))

/Users/paytonsimmons/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/more_dfply/facets.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return col.str.contains(pattern, case=case, regex=regex, na=na)


,Store
61,"Walmart Supercenter; #2070,"
62,"Walmart Supercenter; #4359,"
63,"Walmart Supercenter; #2071,"
64,"Walmart Supercenter; #2188,"
65,"Walmart Supercenter; #2722,"
...,...
6809,"Walmart Supercenter; #1485,"
6810,"Walmart Supercenter; #1412,"
6812,"Walmart Supercenter; #1457,"
6813,"Walmart Supercenter; #1461,"


1.  Some of the address columns contain `(NOP)` to indicated *No overnight parking*.  Extract this information into a new indicator column, then remove it from the address column.

In [17]:
WalParking2 = (walmart
>> mutate(Overnight_parking = case_when((text_filter(X.Address,'(NOP)',regex=False),'No'),
                                        (True,'Yes')))
>> mutate(phone = case_when((text_filter(X.Address,'\W\d{3}\W(\s)?\d{3}-\d{3}', regex=True),X.Address.str.split(",").str.get(4)),
                ((text_filter(X.Address,'^510',regex=True),X.Address.str.split(",").str.get(4))),
                 (True,X.Address.str.split(",").str.get(3))))
>> filter_by(text_filter(X.Address, '^510 Ave', regex=True))) 
WalParking2.head()


/Users/paytonsimmons/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/more_dfply/facets.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return col.str.contains(pattern, case=case, regex=regex, na=na)


,Lat,Lon,Store,Address,Overnight_parking,phone
1921,-95.371245,42.023857,"Walmart Supercenter; #1732,","510 Ave C,Denison,IA,51442,(712_263-2000,",Yes,(712_263-2000


In [18]:
(WalParking2
>> mutate(NOP = ifelse(text_filter(X.Address, 'NOP'), 1, 0))
>> head()
)

,Lat,Lon,Store,Address,Overnight_parking,phone,NOP
1921,-95.371245,42.023857,"Walmart Supercenter; #1732,","510 Ave C,Denison,IA,51442,(712_263-2000,",Yes,(712_263-2000,NaN


2.  The address column contains the phone number of most of the stores.  Extract this information into a new column.  There are some issue with different patterns, so use the divide-and-conquer approach is advised.

In [19]:
mutate_phone_number = X.Address.str.extract('(\(\d\d\d\) \d\d\d-\d\d\d\d)')
mutate_address = X.Address.replace('(\(\d\d\d\) \d\d\d-\d\d\d\d)','',regex=True)

WalParking3 = (walmart
>> mutate(Phone_Number = mutate_phone_number)
>> mutate(Address = mutate_address))
WalParking3.Phone_Number.unique()

array(['(403) 945-1295', '(403) 793-2111', '(403) 730-0990', ...,
       '(307) 362-1957', '(307) 674-6492', '(867) 667-2652'], dtype=object)

3.  Extract the country from the address column.

In [20]:
walmart4 = (WalParking3
>> mutate(Country = case_when((text_filter(X.Address, '[A-Z]{2},\d{4,5}', regex=True), "United States"),
                            (text_filter(X.Address, '[A-Z]\d[A-Z]\s?\d?[A-Z]?\d?', regex=True), "Canada")
                            ))
)
walmart4


,Lat,Lon,Store,Address,Phone_Number,Country
0,-114.005671,51.262567,"Walmart Supercentre; #1050,","2881 Main St SW,Airdrie ,AB T4B 3G5,",(403) 945-1295,Canada
1,-111.900542,50.577939,"Walmart Supercentre; #3658,","917 3rd St W,Brooks ,AB T1R 1L5,",(403) 793-2111,Canada
2,-114.039133,51.107253,"Walmart Supercentre; #3013,","1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,",(403) 730-0990,Canada
3,-114.138488,51.040871,"Walmart Supercentre; #3009,Gas,","1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,",(403) 242-2205,Canada
4,-114.028603,50.930551,"Walmart; #1144,","1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,",(403) 225-6638,Canada
...,...,...,...,...,...,...
6811,-107.209281,41.792084,"Walmart Supercenter; #4471,Gas,","2390 E Cedar St; I-80 Exit 214,Rawlins,WY,8230...",(307) 417-3001,United States
6812,-108.379227,43.042858,"Walmart Supercenter; #1457,","1733 N Federal Blvd,Riverton,WY,82501 ,,",(307) 856-3261,United States
6813,-109.251020,41.579761,"Walmart Supercenter; #1461,","201 Gateway Blvd; I-80 Exit 102,Rock Springs,W...",(307) 362-1957,United States
6814,-106.940967,44.779474,"Walmart Supercenter; #1508,","1695 Coffeen Ave; I-90 Exit 25,Sheridan,WY,828...",(307) 674-6492,United States


4.  Extract the state or province from the address columns

In [21]:
canada = r',AB|,BC|,MB|,NB|,NL|,NT|,NS|,NU|,ON|,PE|,QC|,SK|,YT'
catch_canada = r',(AB|,BC|,MB|,NB|,NL|,NT|,NS|,NU|,ON|,PE|,QC|,SK|,YT)'
us = r',[A-Z]{2},'
catch_us = r',([A-Z]{2}),'
canadian_province = text_filter(X.Address, canada, regex=True, case=True)
walmart5 = (walmart4
>> mutate(S_or_P = case_when((canadian_province, X.Address.str.extract(catch_canada, expand=False)),
                            (True, X.Address.str.extract(catch_us, expand=False))))
)
walmart5

,Lat,Lon,Store,Address,Phone_Number,Country,S_or_P
0,-114.005671,51.262567,"Walmart Supercentre; #1050,","2881 Main St SW,Airdrie ,AB T4B 3G5,",(403) 945-1295,Canada,AB
1,-111.900542,50.577939,"Walmart Supercentre; #3658,","917 3rd St W,Brooks ,AB T1R 1L5,",(403) 793-2111,Canada,AB
2,-114.039133,51.107253,"Walmart Supercentre; #3013,","1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,",(403) 730-0990,Canada,AB
3,-114.138488,51.040871,"Walmart Supercentre; #3009,Gas,","1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,",(403) 242-2205,Canada,AB
4,-114.028603,50.930551,"Walmart; #1144,","1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,",(403) 225-6638,Canada,AB
...,...,...,...,...,...,...,...
6811,-107.209281,41.792084,"Walmart Supercenter; #4471,Gas,","2390 E Cedar St; I-80 Exit 214,Rawlins,WY,8230...",(307) 417-3001,United States,WY
6812,-108.379227,43.042858,"Walmart Supercenter; #1457,","1733 N Federal Blvd,Riverton,WY,82501 ,,",(307) 856-3261,United States,WY
6813,-109.251020,41.579761,"Walmart Supercenter; #1461,","201 Gateway Blvd; I-80 Exit 102,Rock Springs,W...",(307) 362-1957,United States,WY
6814,-106.940967,44.779474,"Walmart Supercenter; #1508,","1695 Coffeen Ave; I-90 Exit 25,Sheridan,WY,828...",(307) 674-6492,United States,WY


5. Combine all of your transformations into one pipe, then re-factor your code to be more readable.

In [24]:
(walmart
>> mutate(Fuel = fuel_case)
>> mutate(Store = store_replace('Gas(/Diesel)?,'))
>> mutate(Storetype = storetype_case)
>> mutate(Storello = storello_case)
>> mutate(Overnight_parking = overnight_parking_case)
>> mutate(Phone = phone_case)
>> mutate(NOP = NOP_ifelse)
>> mutate(Phone_Number = mutate_phone_number)
>> mutate(Address = mutate_address)
>> mutate(Country = country_case)
>> mutate(S_or_P = S_or_P_case)
)


/Users/paytonsimmons/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/more_dfply/facets.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return col.str.contains(pattern, case=case, regex=regex, na=na)
/Users/paytonsimmons/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/more_dfply/facets.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return col.str.contains(pattern, case=case, regex=regex, na=na)
/Users/paytonsimmons/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/more_dfply/facets.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return col.str.contains(pattern, case=case, regex=regex, na=na)
/Users/paytonsimmons/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packa

,Lat,Lon,Store,Address,Fuel,Storetype,Storello,Overnight_parking,Phone,NOP,Phone_Number,Country,S_or_P
0,-114.005671,51.262567,"Walmart Supercentre; #1050,","2881 Main St SW,Airdrie ,AB T4B 3G5,",None,Walmart Supercenter,1050,Yes,(403) 945-1295,0,(403) 945-1295,Canada,AB
1,-111.900542,50.577939,"Walmart Supercentre; #3658,","917 3rd St W,Brooks ,AB T1R 1L5,",None,Walmart Supercenter,3658,Yes,(403) 793-2111,0,(403) 793-2111,Canada,AB
2,-114.039133,51.107253,"Walmart Supercentre; #3013,","1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,",None,Walmart Supercenter,3013,No,(403) 730-0990,1,(403) 730-0990,Canada,AB
3,-114.138488,51.040871,"Walmart Supercentre; #3009,","1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,",Gas,Walmart Supercenter,3009,No,(403) 242-2205,1,(403) 242-2205,Canada,AB
4,-114.028603,50.930551,"Walmart; #1144,","1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,",None,Walmart,1144,Yes,(403) 225-6638,0,(403) 225-6638,Canada,AB
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6811,-107.209281,41.792084,"Walmart Supercenter; #4471,","2390 E Cedar St; I-80 Exit 214,Rawlins,WY,8230...",Gas,Walmart Supercenter,4471,Yes,,0,(307) 417-3001,United States,WY
6812,-108.379227,43.042858,"Walmart Supercenter; #1457,","1733 N Federal Blvd,Riverton,WY,82501 ,,",None,Walmart Supercenter,1457,Yes,,0,(307) 856-3261,United States,WY
6813,-109.251020,41.579761,"Walmart Supercenter; #1461,","201 Gateway Blvd; I-80 Exit 102,Rock Springs,W...",None,Walmart Supercenter,1461,Yes,,0,(307) 362-1957,United States,WY
6814,-106.940967,44.779474,"Walmart Supercenter; #1508,","1695 Coffeen Ave; I-90 Exit 25,Sheridan,WY,828...",None,Walmart Supercenter,1508,No,(NOP),1,(307) 674-6492,United States,WY


In [23]:
new_store_name = {'murphy: USA':'Murphy','Murphy: USA':'Murphy','Walmart Supercentre':'Walmart Supercenter','WM Nbrhd Mkt':'Walmart Neighborhood Market',
            'Wm Nbrhd Mkt':'Walmart Neighborhood Market','wm Nbrhd Mkt':'Walmart Neighborhood Market','Wm Pharmacy/Clinic':'Walmart Pharmacy/Clinic',
            'Walmart':'Walmart',"Sam's Club":'Sams Club','Walmart Pickup & Fuel':'Walmart Pickup & Fuel','Walmart Pickup':'Walmart Pickup',
            'Walmart Fuel Center':'Walmart Fuel Center','Walmart Supercenter':'Walmart Supercenter'}
fuel_case = case_when((text_filter(X.Store, 'Gas/Diesel'),
                                                "Gas/Diesel"),
                          (text_filter(X.Store, 'Gas'),
                                                "Gas"),
                                                (True, "None"))
store_replace = lambda sub: X.Store.replace(sub, '', regex = True)
storetype_case = case_when((text_filter(X['Store'], 'Walmart( Supercentre| Supercenter| Pickup(& Fuel)?)?;(\s)?#\d{3,4},',regex=True),
                      recode(X.Store.str.split(";").str.get(0), new_store_name)),
                      (text_filter(X['Store'],'USA; #\d{4},$',regex=True),
                      recode(X.Store.str.split(";").str.get(0), new_store_name)),
                      (text_filter(X['Store'],'Wm (Nbrhd Mkt|Pharmacy/Clinic)(; |,)#\d{4},',regex=True),
                      recode(X.Store.str.split(";|,").str.get(0), new_store_name)),
                      (text_filter(X['Store'],'\w{3}\W\w Club; #\d{4},', regex=True),
                      recode(X.Store.str.split(";").str.get(0), new_store_name)),
                      (text_filter(X['Store'],'Walmart (Fuel Center|Supercenter),#\d{4},', regex=True),
                      recode(X.Store.str.split(";").str.get(0), new_store_name)),
                      (True,recode(X.Store.str.split(";").str.get(0), new_store_name)))
storello_case = case_when((text_filter(X['Store'],'#\d{3,4},$', regex=True),
          X.Store.str.split("#").str.get(1).str.slice(stop=4)),
          (True,'fudge'))
overnight_parking_case = case_when((text_filter(X.Address,'(NOP)',regex=False),'No'),
                                        (True,'Yes'))
phone_case = case_when((text_filter(X.Address,'\W\d{3}\W(\s)?\d{3}-\d{3}', regex=True),X.Address.str.split(",").str.get(4)),
                ((text_filter(X.Address,'^510',regex=True),X.Address.str.split(",").str.get(4))),
                 (True,X.Address.str.split(",").str.get(3)))
NOP_ifelse = ifelse(text_filter(X.Address, 'NOP'), 1, 0)
mutate_phone_number = X.Address.str.extract('(\(\d\d\d\) \d\d\d-\d\d\d\d)')
mutate_address = X.Address.replace('(\(\d\d\d\) \d\d\d-\d\d\d\d)','',regex=True)
country_case = case_when((text_filter(X.Address, '[A-Z]{2},\d{4,5}', regex=True), "United States"),
                            (text_filter(X.Address, '[A-Z]\d[A-Z]\s?\d?[A-Z]?\d?', regex=True), "Canada")
                            )
canada = r',AB|,BC|,MB|,NB|,NL|,NT|,NS|,NU|,ON|,PE|,QC|,SK|,YT'
catch_canada = r',(AB|,BC|,MB|,NB|,NL|,NT|,NS|,NU|,ON|,PE|,QC|,SK|,YT)'
us = r',[A-Z]{2},'
catch_us = r',([A-Z]{2}),'
canadian_province = text_filter(X.Address, canada, regex=True, case=True)
S_or_P_case = case_when((canadian_province, X.Address.str.extract(catch_canada, expand=False)),
                            (True, X.Address.str.extract(catch_us, expand=False)))

In [25]:
walmart.to_csv("Walmart_Data")